# Projeto de ETL

In [50]:
import pandas as pd
import pandera as pa
import numpy as np
from IPython.display import display

## Extração

In [51]:
path = "ocorrencia.csv"
df = pd.read_csv(path,
                 delimiter=";",
                 parse_dates = ['ocorrencia_dia'],
                 dayfirst = True)  # Primeiro valor é o dia
# na_values

### Schema de dados

In [52]:
HORA_REGEX = fr'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'

schema = pa.DataFrameSchema(
    columns = {
        "codigo_ocorrencia":pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        "ocorrencia_uf":pa.Column(pa.String,
                                  pa.Check.str_length(2,2),  # Aceita somente strings com 2 caracteres
                                  nullable=True),  
        "ocorrencia_aerodromo":pa.Column(pa.String,
                                         nullable=True),
        "ocorrencia_dia":pa.Column(pa.DateTime),
        "ocorrencia_hora":pa.Column(pa.String,
                                    pa.Check.str_matches(HORA_REGEX),
                                    nullable=True),         # usar regex
        "total_recomendacoes":pa.Column(pa.Int)
    }
)

In [53]:
# Filtrar apenas colunas relevantes para nosso ETL
df = df[list(schema.columns)]
df.head(5)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,****,2012-01-06,13:00:00,0
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,****,2012-01-06,17:00:00,0
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,****,2012-01-06,16:30:00,0


In [54]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

## Limpeza

In [55]:
def checar_dados(serie: pd.Series):
    print("Quantidade de valores únicos :", serie.nunique())
    print("Quantidade de registros nulos :", serie.isna().sum())
    print("Lista de valores únicos :")
    display(serie.unique().tolist()) 

In [56]:
duplicatas = df.duplicated().sum()
if  duplicatas != 0:
    print("Dropando duplicatas... {duplicatas} registros duplicados")
    df.drop_duplicates(inplace=True)

else:
    print("Não há registros duplicados")
# Não há registros duplicados

Não há registros duplicados


In [57]:
# Campos e respectivos valores a serem tratados:
CLEAN_COLS = {}

for col in df.columns:
    if df[col].dtype == object:
        print(f'{col}: ', df[col].dtype)

ocorrencia_classificacao:  object
ocorrencia_cidade:  object
ocorrencia_uf:  object
ocorrencia_aerodromo:  object
ocorrencia_hora:  object


In [58]:
checar_dados(df['ocorrencia_classificacao'])
# Tudo OK

Quantidade de valores únicos : 3
Quantidade de registros nulos : 0
Lista de valores únicos :


['INCIDENTE', 'ACIDENTE', 'INCIDENTE GRAVE']

In [59]:
checar_dados(df['ocorrencia_cidade'])
# Verificação visual dos dados no editor de texto
# Tudo OK

Quantidade de valores únicos : 1053
Quantidade de registros nulos : 0
Lista de valores únicos :


['PORTO ALEGRE',
 'GUARULHOS',
 'VIAMÃO',
 'SÃO SEBASTIÃO',
 'SÃO SEPÉ',
 'UBATUBA',
 'CAMPINAS',
 'BELÉM',
 'CONCEIÇÃO DAS ALAGOAS',
 'UBERLÂNDIA',
 'MARABÁ',
 'ELDORADO',
 'RIO BRANCO',
 'CÁCERES',
 'MACAÉ',
 'BRASÍLIA',
 'CAMOCIM',
 'MUANÁ',
 'BELO HORIZONTE',
 'CACHOEIRA DOURADA',
 'CUIABÁ',
 'RONDONÓPOLIS',
 'BRAGANÇA PAULISTA',
 'SÃO PAULO',
 'RIO DE JANEIRO',
 'RECIFE',
 'TASSO FRAGOSO',
 'ITAPURANGA',
 'VITÓRIA',
 'RIBEIRÃO PRETO',
 'JOINVILLE',
 'CAMPOS LINDOS',
 'PELOTAS',
 'NOVA BRASILÂNDIA',
 'VÁRZEA GRANDE',
 'CAMPOS DOS GOYTACAZES',
 'CAMPO FLORIDO',
 'SÃO JOSÉ DOS PINHAIS',
 'CAMPO GRANDE',
 'CORRENTINA',
 'LONDRINA',
 'PIRACICABA',
 'PALESTINA',
 'ITANHAÉM',
 'JUNDIAÍ',
 'QUELUZITO',
 'SONORA',
 'BELA VISTA DO PARAÍSO',
 'ARROIO GRANDE',
 'CAIRU',
 'CAMETÁ',
 'PERUÍBE',
 'CABO FRIO',
 'ESTÂNCIA',
 'MANAUS',
 'MARITUBA',
 'CORUPÁ',
 'COSTA RICA',
 'CONFINS',
 'SÃO JOSÉ',
 'ENVIRA',
 'LUIZ ALVES',
 'BARRA DO GARÇAS',
 'QUERÊNCIA DO NORTE',
 'LINS',
 'FORTALEZA',
 'SÃO JOS

In [60]:
checar_dados(df['ocorrencia_uf'])
# Verificação visual dos dados no editor de texto
# Valor "***" que não corresponde a nenhuma UF; subsituir por NÃO INFORMADO
CLEAN_COLS['ocorrencia_uf'] = ["***"]

Quantidade de valores únicos : 28
Quantidade de registros nulos : 0
Lista de valores únicos :


['RS',
 'SP',
 'PA',
 'MG',
 'AC',
 'MT',
 'RJ',
 'DF',
 'CE',
 'GO',
 'PE',
 'MA',
 'ES',
 'SC',
 'TO',
 'PR',
 'MS',
 'BA',
 'SE',
 'AM',
 'PI',
 'AL',
 'AP',
 'RR',
 'RO',
 'RN',
 'PB',
 '***']

In [61]:
checar_dados(df['ocorrencia_aerodromo'])
# Verificação visual dos dados no editor de texto
# Valor "***" que não corresponde a nenhuma UF; subsituir por NÃO INFORMADO
CLEAN_COLS['ocorrencia_aerodromo'] = ["###!", "####", "****", "*****"]

Quantidade de valores únicos : 516
Quantidade de registros nulos : 0
Lista de valores únicos :


['SBPA',
 'SBGR',
 '****',
 'SDAI',
 'SBBE',
 'SBUL',
 'SBMA',
 'SBBR',
 'SNWC',
 'SBPR',
 'SWRD',
 'SDVH',
 'SBMT',
 'SBJR',
 'SBRF',
 'SBVT',
 'SBBH',
 'SBJV',
 'SBCT',
 'SBCG',
 'SBGL',
 'SBLO',
 'SDPW',
 'SBSP',
 'SBJD',
 'SDAM',
 'SNCL',
 'SNOD',
 'SBBP',
 'SWFN',
 'SBEG',
 'SBCF',
 'SSKT',
 'SBBW',
 'SWXQ',
 'SBFZ',
 'SBRB',
 'SBRJ',
 'SBSV',
 'SBPJ',
 'SBME',
 'SBRP',
 'SBIL',
 'SBBZ',
 'SDCO',
 'SBGO',
 'SDMC',
 'SBNF',
 'SBMO',
 'SBBI',
 'SBNV',
 'SNOF',
 'SNRU',
 'SBBU',
 'SSKG',
 'SBGW',
 'SSBL',
 'SBMQ',
 'SSXO',
 'SNTF',
 'SBMG',
 'SWMV',
 'SWFR',
 'SSCR',
 'SBUR',
 'SBSR',
 'SNAZ',
 'SBNT',
 'SBKP',
 'SDBK',
 'SBTD',
 'SDBN',
 'SBCA',
 'SDTF',
 'SBPG',
 'SBJC',
 'SDRK',
 'SBCB',
 'SBCY',
 'SJHR',
 'SDAA',
 'SWEK',
 'SIQE',
 'SBSJ',
 'SSOS',
 'SBFI',
 'SILT',
 'SBBV',
 'SSDO',
 'SDBB',
 'SBCJ',
 'SSNB',
 'SBPS',
 'SBMS',
 'SWKO',
 'SDIO',
 'SBUY',
 'SDTB',
 'SBTE',
 'SWUI',
 'SBSN',
 'SNKB',
 'SSBN',
 'SBDN',
 'SDXF',
 'SNDH',
 'SJPT',
 'SSOK',
 'SSHS',
 'SSSB',
 'SBCP',
 

In [62]:
checar_dados(df['ocorrencia_hora'])
# Verificação visual dos dados no editor de texto
# Valor nulo encontrado

Quantidade de valores únicos : 923
Quantidade de registros nulos : 1
Lista de valores únicos :


['20:27:00',
 '13:44:00',
 '13:00:00',
 '17:00:00',
 '16:30:00',
 '14:30:00',
 '18:15:00',
 '19:12:00',
 '16:00:00',
 '22:13:00',
 '13:30:00',
 '13:36:00',
 '11:21:00',
 '13:45:00',
 '10:00:00',
 '10:30:00',
 '16:47:00',
 '18:25:00',
 '17:35:00',
 '13:10:00',
 '21:30:00',
 '21:00:00',
 '08:00:00',
 '14:50:00',
 '23:30:00',
 '20:45:00',
 '11:15:00',
 '20:25:00',
 '19:05:00',
 '23:00:00',
 '09:33:00',
 '22:00:00',
 '09:30:00',
 '14:45:00',
 '18:35:00',
 '13:55:00',
 '18:00:00',
 '14:15:00',
 '12:30:00',
 '13:50:00',
 '22:35:00',
 '23:33:00',
 '02:04:00',
 '14:55:00',
 '17:34:00',
 '19:30:00',
 '14:00:00',
 '18:44:00',
 '20:00:00',
 '16:10:00',
 '20:56:00',
 '20:50:00',
 '19:45:00',
 '01:44:00',
 '09:50:00',
 '21:35:00',
 '22:05:00',
 '04:00:00',
 '19:25:00',
 '02:20:00',
 '13:20:00',
 '12:00:00',
 '11:30:00',
 '15:10:00',
 '14:10:00',
 '09:10:00',
 '13:40:00',
 '22:45:00',
 '19:35:00',
 '10:25:00',
 '11:45:00',
 '20:04:00',
 '11:47:00',
 '15:42:00',
 '19:15:00',
 '10:15:00',
 '12:50:00',

In [63]:
# Ocorrências para Limpeza
CLEAN_COLS

{'ocorrencia_uf': ['***'],
 'ocorrencia_aerodromo': ['###!', '####', '****', '*****']}

In [64]:
for key in CLEAN_COLS.keys():
    df[key] = np.where(df[key].isin(CLEAN_COLS[key]),
                       pd.NA,
                       df[key])

In [65]:
# Tratamento específico para dados de sigla de UF
df['ocorrencia_uf'] = np.where(df['ocorrencia_uf'].str.len() != 2, # Condição específica do campo
                               pd.NA, # Se Sim
                               df['ocorrencia_uf']) # Se Não

In [66]:
df.isna().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  2
ocorrencia_aerodromo        1905
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [67]:
fill_values = {
    'ocorrencia_aerodromo':'NI',
    'ocorrencia_uf':'NI',
}

df.fillna(value= fill_values, inplace=True)  # Para classificar dados não informados

## Validação

In [68]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,NI,2012-01-06,13:00:00,0
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,NI,2012-01-06,17:00:00,0
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,NI,2012-01-06,16:30:00,0
...,...,...,...,...,...,...,...,...,...
5162,80458,80458,ACIDENTE,JATAÍ,GO,NI,2021-12-30,20:30:00,0
5163,80452,80452,ACIDENTE,MARACAÍ,SP,NI,2021-12-31,09:30:00,0
5164,80457,80457,INCIDENTE GRAVE,NOVO HAMBURGO,RS,SSNH,2021-12-31,11:59:00,0
5165,80460,80460,INCIDENTE,CURITIBA,PR,SBBI,2021-12-31,15:12:00,0


## Transformação

In [70]:
mask = ((df.ocorrencia_uf == 'NI'))

df[mask]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
1099,49474,49474,ACIDENTE,ÁGUAS INTERNACIONAIS,NI,NI,2013-09-02,02:54:00,0
4728,79956,79956,ACIDENTE,NÃO IDENTIFICADA,NI,NI,2021-02-26,11:00:00,0


In [73]:
mask = (df.ocorrencia_classificacao == 'INCIDENTE GRAVE') & ((df.ocorrencia_uf == 'AM'))

df[mask].groupby(['ocorrencia_aerodromo', 'ocorrencia_cidade']).size()


ocorrencia_aerodromo  ocorrencia_cidade    
NI                    CANUTAMA                 1
                      COARI                    1
                      IPIXUNA                  1
                      ITAPIRANGA               1
                      NOVO ARIPUANÃ            1
                      PRESIDENTE FIGUEIREDO    1
SBEG                  MANAUS                   2
SBTF                  TEFÉ                     2
SWCA                  CARAUARI                 2
SWFN                  MANAUS                   6
SWJP                  JAPURÁ                   1
SWKO                  COARI                    1
SWMW                  MAUÉS                    1
SWOB                  FONTE BOA                1
dtype: int64